## CNN

원래 이미지 인식에 탁월한 효과를 보이는 딥러닝 신경망이지만, 문서 분류에 뛰어난 효과를 보이는 것으로 입증되어 텍스트 마이닝 분야에서도
활발하게 쓰이는 모형이 되었다.
BERT의 결과와 CNN을 결합한 모형이 제안되는 등 여전히 많이 사용된다.

CNN : 2차원 행렬로부터 주변 정보를 요약해 이미지를 분류할 수 있는 특성들을 추출하는 딥러닝 기법이다. (컬러 이미지 : 3차원 행렬)

**CNN 구조**
1 . 컨볼루션 단계
원본 이미지에 필터를 적용해 주변 정보를 요약하는 단계
필터는 원본 이미지보다 작은 2차원 행렬로 학습의 대상이 되는 파라미터 (파라미터에 따라 요약정보는 달라짐. weight의 느낌)
이 단계에서 중요한 요소는 필터 혹은 채너의 수이다.
필터를 2개를 사용한다면, 두 개의 요약된 2차원 행렬을 만들어낼 수 있다. (하나의 이미지 -> 2가지의 다른 정보 추출)
2 . 풀링 단계
요약된 2차원 행렬을 축소하는 단계
max pooling의 경우 각 필터 범위 안에 있는 가장 높은 값을 추출해내 새로운 2차원 행렬을 만든다. (이미지 너비 축소)

이 특성을 통해 이미지의 정보를 축략한 특성들을 만들어내고, 모형의 마지막 단계에서 이 특성을 가지고 이미지를 판별한다.

### CNN을 이용한 문서 분류

텍스트 마이닝에 딥러닝을 적용하는 이유는 **문맥 파악**이며, CNN을 사용하는 이유는 단어들의 연속된 나열에 대해 앞 뒤 단어들 간의 주변정보를 요약해낼 수 있다는 것이다.

문서에서는 단어들의 나열을 보기 때문에 **2차원이 아닌 1차원 방향으로만** 필터를 이동시키면서 컨볼루션을 수행한다.
(옆/아래로 이동이 아닌 아래로만 이동)

문서는 한 단어가 하나의 행으로 반환되어 단어들은 순서에 따라 **아래방향**으로 나열된다.
여기서 단어들 간의 주변 정보를 요약하기 위해서 필터의 열 수를 단어 벡터의 수와 같게 하고(단어의 개수 -> dictionary에 있는 단어의 전체 개수/ 혹은 밀집벡터)
단어의 나열 방향에 따라 필터를 **아래**로만 이동하며 주변 정보를 요약한다.

In [1]:
### CNN을 이용한 NLTK 영화 리뷰
from nltk.corpus import movie_reviews
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

fileids = movie_reviews.fileids()
reviews = [movie_reviews.raw(fileids) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

np.random.seed(7)
tf.random.set_seed(7)

max_words = 1000
maxlen = 500

tokenizer = Tokenizer(num_words = max_words, oov_token = "UNK")
tokenizer.fit_on_texts(reviews) #단어 인덱스 구축

X = tokenizer.texts_to_sequences(reviews) #만들어진 단어 인덱스를 이용해 변환
X= pad_sequences(X, maxlen = maxlen, truncating = 'pre')

label_dict = {'pos':0, 'neg':1}
y = np.array([label_dict[c] for c in categories])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 10)

MemoryError: 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Embedding, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential([
    #word embedding layer
    Embedding(max_words, 64, input_length = maxlen),   #500X64의 행렬로 변환 (500은 maxlen)
    Conv1D(128, #채널의 수 128개
           5, #필터의 크기 5x5
           padding = 'valid', #패딩 없음
           activation = 'relu',
           strides = 1), #필터 이동 간격 : 1
    MaxPooling1D(),
    Conv1D(256, #채널의 수를 256개로 늘림
           5,
           padding = 'valid',
           activation = 'relu',
           strides = 1),
    MaxPooling1D(),
    Flatten(),
    Dense(64, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

model.summary()

In [ ]:
adam = Adam(learning_rate = 1e-3)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(X_train, y_train,
                    epochs = 20,
                    batch_size = 256,
                    verbose = 0,
                    validation_split = 0.2)

%matplotlib inline
import matplotlib.pyplot as plt

def plot_results(history, metric):
    plt.plot(history.history[metric], 'b', label = 'Training ' + metric)
    plt.plot(history.history['val_'+metric], 'r--', label = 'validation '+ metric)
    plt.title('Training. vs Validation. '+metric)
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend()
    plt.show()

plot_results(history, 'loss')

score = model.evaluate(X_test, y_test)
print(f"Test accuracy : {score[1]:.3f}")

#### 결과 분석
500개나 되는 단어들로 문서가 이루어져 있을 때, 그 500개의 단어들을 다 연결해 문맥을 파악하는 것이 더 효율적인지, 아니면 5개의 단어 정도로 끊어가며 좁은 범위에서 문맥을 파악하고 그 결과를 합치는 것이 더 효율적인지는 알 수 없다.